In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import math

from utils import load_and_preprocess_data

from PositionalEncoding import PositionalEncoding
from MultiHeadAttention import MHA

# Load and Preprocess data

In [2]:
data, vocab  = load_and_preprocess_data()

tokenize = {char : index for index, char in enumerate(vocab) }
detokenize = {char : index for char, index in enumerate (list(tokenize.keys()))}

tokens = []

for char in data:
    tokens.append(tokenize[char])

num_embeddings = len(tokenize.keys())
embedding_dim = 50

vocab_size = len(tokenize.keys())

# Begin Composing model

In [3]:
class Model(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim)
        self.pos_enc = PositionalEncoding(d_model=embedding_dim, max_len=50)
        self.mha = MHA(d_model=embedding_dim, n_heads=5)

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_enc(x)
        out, attn_weights = self.mha(x)
        return out
        
        

In [4]:
model = Model(num_embeddings, embedding_dim)

In [5]:
x = torch.randint(0, 9, (1, 7))
o = model(x)
o.shape

torch.Size([1, 7, 50])

# Work in Progress below (Add & Norm)

In [7]:
class AddNorm(nn.Module):
    def __init__(self):

        return